In [1]:
"""
Numerical Parser for Financial QA Evaluation
==============================================

This module provides robust numerical extraction from text answers in financial QA systems.
Handles currency symbols, percentages, scales (million/billion), negatives, and various formats.

Author: Financial QA Evaluation System
Version: 1.0
"""

import re
from typing import Optional, Dict, Any, Tuple
from enum import Enum


class NumberScale(Enum):
    """Enumeration for number scales"""
    NONE = 1
    THOUSAND = 1_000
    MILLION = 1_000_000
    BILLION = 1_000_000_000
    TRILLION = 1_000_000_000_000


def extract_number(text: str, return_metadata: bool = False) -> Optional[float] | Dict[str, Any]:
    """
    Extract numerical value from text with various formats.
    
    This function handles:
    - Currency symbols: "$1577.00", "USD 1577", "1577 dollars"
    - Percentage signs: "24.5%", "24.5 percent"
    - Negative numbers: "-3.7", "negative 3.7"
    - Thousands separators: "1,577" vs "1577"
    - Scale indicators: "1577 million", "1.577 billion"
    - Decimal variations: "1577.0", "1577.00"
    - Text with qualifiers: "approximately $1,400 million", "about 24.5%"
    - Percentage as decimals: "0.8" (interpreted as 80% if magnitude < 1)
    
    Args:
        text: Input text potentially containing a number
        return_metadata: If True, return dict with number, scale, is_percentage, and raw_text
                        If False, return just the normalized float value
    
    Returns:
        If return_metadata=False: 
            Float value (normalized to base unit) or None if no number found
        If return_metadata=True:
            Dict with keys: 'value', 'scale', 'scale_factor', 'is_percentage', 'raw_text', 'normalized_value'
    
    Examples:
        >>> extract_number("$1577.00")
        1577.0
        
        >>> extract_number("24.5%")
        24.5
        
        >>> extract_number("-3.7")
        -3.7
        
        >>> extract_number("approximately $1,400 million")
        1400.0
        
        >>> extract_number("The answer is 0.96")
        0.96
        
        >>> extract_number("I don't know")
        None
        
        >>> extract_number("1,577.00")
        1577.0
        
        >>> extract_number("$1.577 billion")
        1.577
        
        >>> extract_number("0.8")  # Interpreted as decimal if no context
        0.8
        
        >>> extract_number("78.95%")
        78.95
        
        >>> extract_number("Data not available.")
        None
    """
    
    if not text or not isinstance(text, str):
        return None if not return_metadata else {
            'value': None, 'scale': None, 'scale_factor': 1, 
            'is_percentage': False, 'raw_text': text, 'normalized_value': None
        }
    
    # Normalize text
    text_lower = text.lower().strip()
    
    # Check for explicit refusal patterns (return None early)
    refusal_patterns = [
        r'data not available',
        r'cannot (be )?(calculate|determine|compute)',
        r'insufficient (information|data)',
        r'not applicable',
        r'n/?a\b',
        r'no (data|information) available'
    ]
    
    for pattern in refusal_patterns:
        if re.search(pattern, text_lower):
            return None if not return_metadata else {
                'value': None, 'scale': None, 'scale_factor': 1,
                'is_percentage': False, 'raw_text': text, 'normalized_value': None
            }
    
    # Detect percentage
    is_percentage = bool(re.search(r'%|percent|percentage', text_lower))
    
    # Detect scale (million, billion, etc.)
    scale_pattern = r'\b(thousand|million|billion|trillion)s?\b'
    scale_match = re.search(scale_pattern, text_lower)
    
    scale_str = None
    scale_factor = 1
    
    if scale_match:
        scale_str = scale_match.group(1)
        if scale_str == 'thousand':
            scale_factor = NumberScale.THOUSAND.value
        elif scale_str == 'million':
            scale_factor = NumberScale.MILLION.value
        elif scale_str == 'billion':
            scale_factor = NumberScale.BILLION.value
        elif scale_str == 'trillion':
            scale_factor = NumberScale.TRILLION.value
    
    # Main number extraction pattern
    # Handles: optional negative, optional $, digits with optional commas, optional decimal part
    # Pattern matches either:
    # 1. Numbers with commas: -?$?\s*\d{1,3}(?:,\d{3})+(?:\.\d+)?
    # 2. Numbers without commas: -?$?\s*\d+(?:\.\d+)?
    number_pattern = r'-?\$?\s*\d{1,3}(?:,\d{3})+(?:\.\d+)?|-?\$?\s*\d+(?:\.\d+)?'
    
    # Find all potential numbers
    matches = re.finditer(number_pattern, text)
    
    numbers_found = []
    for match in matches:
        num_str = match.group()
        # Clean the number string
        num_str = num_str.replace('$', '').replace(',', '').strip()
        
        try:
            num_value = float(num_str)
            numbers_found.append(num_value)
        except ValueError:
            continue
    
    if not numbers_found:
        return None if not return_metadata else {
            'value': None, 'scale': scale_str, 'scale_factor': scale_factor,
            'is_percentage': is_percentage, 'raw_text': text, 'normalized_value': None
        }
    
    # Take the first substantial number (prefer non-zero, but accept zero if it's the only one)
    raw_value = None
    for num in numbers_found:
        if num != 0:
            raw_value = num
            break
    
    # If all numbers are zero, use zero
    if raw_value is None:
        raw_value = numbers_found[0]
    
    # For percentages stored as decimals (0.8 meaning 80%), we need to be smart
    # If the number is between 0 and 1, and it's labeled as a percentage or ratio question,
    # we might need to convert. But based on user requirements:
    # - If text has %, keep as is (78.95% stays 78.95)
    # - If text is 0.8 without %, keep as 0.8 (comparison logic will handle equivalence)
    
    # Don't apply scale factor here - return the base value
    # The comparison function will handle scale normalization
    normalized_value = raw_value  # Keep the raw value without scale multiplication
    
    if return_metadata:
        return {
            'value': raw_value,
            'scale': scale_str,
            'scale_factor': scale_factor,
            'is_percentage': is_percentage,
            'raw_text': text,
            'normalized_value': normalized_value
        }
    
    return normalized_value


def normalize_to_same_scale(
    num1: float, 
    scale1: Optional[str], 
    num2: float, 
    scale2: Optional[str]
) -> Tuple[float, float, Optional[str]]:
    """
    Normalize two numbers to the same scale for comparison.
    
    Args:
        num1: First number
        scale1: Scale of first number ('million', 'billion', etc.)
        num2: Second number  
        scale2: Scale of second number
    
    Returns:
        Tuple of (normalized_num1, normalized_num2, common_scale)
        Both numbers will be in the same unit system for comparison
    
    Examples:
        >>> normalize_to_same_scale(1577, 'million', 1.577, 'billion')
        (1577.0, 1577.0, 'million')
        
        >>> normalize_to_same_scale(1577, None, 1577, 'million')
        (1577.0, 1577.0, None)  # Compare raw values when one scale is missing
    """
    
    # If both have no scale or same scale, return as-is
    if scale1 == scale2:
        return num1, num2, scale1
    
    # If one scale is missing, compare raw numbers
    if scale1 is None or scale2 is None:
        return num1, num2, None
    
    # Get scale factors
    scale_map = {
        'thousand': NumberScale.THOUSAND.value,
        'million': NumberScale.MILLION.value,
        'billion': NumberScale.BILLION.value,
        'trillion': NumberScale.TRILLION.value
    }
    
    factor1 = scale_map.get(scale1, 1)
    factor2 = scale_map.get(scale2, 1)
    
    # Normalize to smaller scale (more precise)
    if factor1 < factor2:
        # Scale 1 is smaller, convert num2 to scale1
        # e.g., num1 is in millions, num2 is in billions
        # Convert billions to millions: 1.2 billion = 1200 million
        num2_normalized = num2 * (factor2 / factor1)
        return num1, num2_normalized, scale1
    elif factor1 > factor2:
        # Scale 2 is smaller, convert num1 to scale2  
        # e.g., num1 is in billions, num2 is in millions
        # Convert billions to millions: 1.2 billion = 1200 million
        num1_normalized = num1 * (factor1 / factor2)
        return num1_normalized, num2, scale2
    else:
        # Same scale
        return num1, num2, scale1


def compare_percentages(
    val1: float, 
    is_pct1: bool, 
    val2: float, 
    is_pct2: bool
) -> Tuple[float, float]:
    """
    Normalize percentage values for comparison.
    
    Handles cases like:
    - 0.8 (as ratio) vs 80% (as percentage) 
    - 24.5% vs 0.245 (as decimal)
    
    Args:
        val1: First value
        is_pct1: Whether first value is marked as percentage
        val2: Second value
        is_pct2: Whether second value is marked as percentage
    
    Returns:
        Tuple of (normalized_val1, normalized_val2) both in same format
    
    Examples:
        >>> compare_percentages(0.8, False, 78.95, True)
        (80.0, 78.95)  # Convert 0.8 to 80% for comparison
        
        >>> compare_percentages(24.5, True, 0.245, False)
        (24.5, 24.5)  # Convert 0.245 to 24.5%
    """
    
    # If both are percentages or both are not, return as-is
    if is_pct1 == is_pct2:
        # Special case: if neither is marked as percentage but values suggest one is decimal form
        # e.g., 0.8 vs 80
        if not is_pct1 and not is_pct2:
            # If one value is in [0, 1] range and the other is > 1, assume decimal form
            if 0 <= val1 <= 1 and val2 > 1:
                # val1 might be decimal form of percentage
                val1_normalized = val1 * 100
                return val1_normalized, val2
            elif 0 <= val2 <= 1 and val1 > 1:
                # val2 might be decimal form of percentage
                val2_normalized = val2 * 100
                return val1, val2_normalized
        
        return val1, val2
    
    # One is percentage, one is not
    # Convert the non-percentage to percentage if it's in decimal form (0-1 range)
    
    if is_pct1 and not is_pct2:
        # val1 is percentage (e.g., 78.95%), val2 is not (e.g., 0.7895)
        if 0 <= val2 <= 1:
            # Likely decimal form, convert to percentage
            val2_normalized = val2 * 100
            return val1, val2_normalized
        else:
            # val2 is already in percentage scale
            return val1, val2
    
    if is_pct2 and not is_pct1:
        # val2 is percentage, val1 is not
        if 0 <= val1 <= 1:
            # Likely decimal form, convert to percentage
            val1_normalized = val1 * 100
            return val1_normalized, val2
        else:
            # val1 is already in percentage scale
            return val1, val2
    
    return val1, val2


# Module-level test function for quick validation
def _test_extract_number():
    """Quick sanity test for extract_number function"""
    
    test_cases = [
        ("$1577.00", 1577.0),
        ("24.5%", 24.5),
        ("-3.7", -3.7),
        ("approximately $1,400 million", 1400.0),
        ("The answer is 0.96", 0.96),
        ("I don't know", None),
        ("1,577.00", 1577.0),
        ("Data not available.", None),
        ("$8.70", 8.70),
        ("93.86", 93.86),
        ("$1616.00", 1616.0),
        ("0.8", 0.8),
        ("78.95%", 78.95),
        ("$0.40", 0.40),
        ("$1.577 billion", 1.577),
        ("8.738", 8.738),
        ("1,615.9 million", 1615.9),
        ("0.389 billion", 0.389),
        ("79.83%", 79.83),
    ]
    
    print("Running quick tests for extract_number()...")
    passed = 0
    failed = 0
    
    for text, expected in test_cases:
        result = extract_number(text)
        if result == expected:
            passed += 1
            print(f"✓ PASS: '{text}' -> {result}")
        else:
            failed += 1
            print(f"✗ FAIL: '{text}' -> Expected {expected}, Got {result}")
    
    print(f"\nResults: {passed} passed, {failed} failed out of {len(test_cases)} tests")
    return failed == 0


if __name__ == "__main__":
    # Run tests
    success = _test_extract_number()
    
    # Show metadata examples
    print("\n" + "="*70)
    print("Metadata Examples:")
    print("="*70)
    
    examples = [
        "$1,577 million",
        "$1.577 billion", 
        "0.8",
        "78.95%",
        "approximately $1,400 million"
    ]
    
    for ex in examples:
        result = extract_number(ex, return_metadata=True)
        print(f"\nText: '{ex}'")
        print(f"  Value: {result['value']}")
        print(f"  Scale: {result['scale']}")
        print(f"  Scale Factor: {result['scale_factor']}")
        print(f"  Is Percentage: {result['is_percentage']}")
        print(f"  Normalized Value: {result['normalized_value']}")

Running quick tests for extract_number()...
✓ PASS: '$1577.00' -> 1577.0
✓ PASS: '24.5%' -> 24.5
✓ PASS: '-3.7' -> -3.7
✓ PASS: 'approximately $1,400 million' -> 1400.0
✓ PASS: 'The answer is 0.96' -> 0.96
✓ PASS: 'I don't know' -> None
✓ PASS: '1,577.00' -> 1577.0
✓ PASS: 'Data not available.' -> None
✓ PASS: '$8.70' -> 8.7
✓ PASS: '93.86' -> 93.86
✓ PASS: '$1616.00' -> 1616.0
✓ PASS: '0.8' -> 0.8
✓ PASS: '78.95%' -> 78.95
✓ PASS: '$0.40' -> 0.4
✓ PASS: '$1.577 billion' -> 1.577
✓ PASS: '8.738' -> 8.738
✓ PASS: '1,615.9 million' -> 1615.9
✓ PASS: '0.389 billion' -> 0.389
✓ PASS: '79.83%' -> 79.83

Results: 19 passed, 0 failed out of 19 tests

Metadata Examples:

Text: '$1,577 million'
  Value: 1577.0
  Scale: million
  Scale Factor: 1000000
  Is Percentage: False
  Normalized Value: 1577.0

Text: '$1.577 billion'
  Value: 1.577
  Scale: billion
  Scale Factor: 1000000000
  Is Percentage: False
  Normalized Value: 1.577

Text: '0.8'
  Value: 0.8
  Scale: None
  Scale Factor: 1
  Is Per

In [2]:
"""
Comprehensive Test Suite for extract_number()
==============================================

Tests cover all edge cases from FinanceBench dataset including:
- Real examples from metrics-generated questions
- Real examples from domain-relevant questions  
- Real examples from novel-generated questions
- Currency formats
- Percentages
- Scales (million, billion)
- Negative numbers
- Refusals
"""

import sys
sys.path.append('/home/claude')


def test_basic_numbers():
    """Test basic number extraction"""
    print("\n" + "="*70)
    print("TEST SUITE 1: Basic Number Extraction")
    print("="*70)
    
    tests = [
        # (input, expected_output, description)
        ("1577", 1577.0, "Plain integer"),
        ("1577.0", 1577.0, "Integer with decimal zero"),
        ("1577.00", 1577.0, "Integer with two decimal zeros"),
        ("8.70", 8.70, "Decimal number"),
        ("93.86", 93.86, "Decimal number"),
        ("0.96", 0.96, "Decimal less than 1"),
        ("0.8", 0.8, "Single decimal digit"),
        ("0", 0.0, "Zero"),
        ("-3.7", -3.7, "Negative decimal"),
        ("-0.02", -0.02, "Negative small decimal"),
    ]
    
    passed, failed = 0, 0
    for text, expected, desc in tests:
        result = extract_number(text)
        if result == expected:
            passed += 1
            print(f"✓ {desc:40} '{text}' -> {result}")
        else:
            failed += 1
            print(f"✗ {desc:40} '{text}' -> Expected {expected}, Got {result}")
    
    print(f"\nBasic Numbers: {passed}/{len(tests)} passed")
    return passed, failed


def test_currency_formats():
    """Test currency symbol handling"""
    print("\n" + "="*70)
    print("TEST SUITE 2: Currency Formats")
    print("="*70)
    
    tests = [
        ("$1577.00", 1577.0, "Dollar sign with decimals"),
        ("$8.70", 8.70, "Small dollar amount"),
        ("$0.40", 0.40, "Less than one dollar"),
        ("$1616.00", 1616.0, "Four digit dollar amount"),
        ("$ 1577", 1577.0, "Dollar with space"),
        ("USD 1577", 1577.0, "USD prefix"),
        ("1577 dollars", 1577.0, "Dollars suffix"),
    ]
    
    passed, failed = 0, 0
    for text, expected, desc in tests:
        result = extract_number(text)
        if result == expected:
            passed += 1
            print(f"✓ {desc:40} '{text}' -> {result}")
        else:
            failed += 1
            print(f"✗ {desc:40} '{text}' -> Expected {expected}, Got {result}")
    
    print(f"\nCurrency Formats: {passed}/{len(tests)} passed")
    return passed, failed


def test_percentages():
    """Test percentage handling"""
    print("\n" + "="*70)
    print("TEST SUITE 3: Percentage Formats")
    print("="*70)
    
    tests = [
        ("24.5%", 24.5, "Percentage with decimal"),
        ("1.9%", 1.9, "Small percentage"),
        ("78.95%", 78.95, "Two decimal percentage"),
        ("79.83%", 79.83, "Two decimal percentage"),
        ("24.5 percent", 24.5, "Word 'percent'"),
        ("100%", 100.0, "Whole percentage"),
        ("-3.7%", -3.7, "Negative percentage"),
    ]
    
    passed, failed = 0, 0
    for text, expected, desc in tests:
        result = extract_number(text)
        if result == expected:
            passed += 1
            print(f"✓ {desc:40} '{text}' -> {result}")
        else:
            failed += 1
            print(f"✗ {desc:40} '{text}' -> Expected {expected}, Got {result}")
    
    print(f"\nPercentages: {passed}/{len(tests)} passed")
    return passed, failed


def test_thousand_separators():
    """Test comma handling in numbers"""
    print("\n" + "="*70)
    print("TEST SUITE 4: Thousand Separators")
    print("="*70)
    
    tests = [
        ("1,577", 1577.0, "Thousands separator"),
        ("1,577.00", 1577.0, "Thousands with decimals"),
        ("$1,400", 1400.0, "Dollar with thousands"),
        ("1,615.9", 1615.9, "Thousands with decimal"),
        ("10,000", 10000.0, "Ten thousand"),
        ("1,000,000", 1000000.0, "One million with commas"),
    ]
    
    passed, failed = 0, 0
    for text, expected, desc in tests:
        result = extract_number(text)
        if result == expected:
            passed += 1
            print(f"✓ {desc:40} '{text}' -> {result}")
        else:
            failed += 1
            print(f"✗ {desc:40} '{text}' -> Expected {expected}, Got {result}")
    
    print(f"\nThousand Separators: {passed}/{len(tests)} passed")
    return passed, failed


def test_scales():
    """Test scale indicators (million, billion)"""
    print("\n" + "="*70)
    print("TEST SUITE 5: Scale Indicators")
    print("="*70)
    
    tests = [
        # (input, expected_value, expected_scale)
        ("1577 million", 1577.0, "million"),
        ("1.577 billion", 1.577, "billion"),
        ("$1,400 million", 1400.0, "million"),
        ("1,615.9 million", 1615.9, "million"),
        ("0.389 billion", 0.389, "billion"),
        ("$1.577 billion", 1.577, "billion"),
        ("15 billion", 15.0, "billion"),
        ("1.2 billion", 1.2, "billion"),
    ]
    
    passed, failed = 0, 0
    for text, expected_value, expected_scale in tests:
        result = extract_number(text, return_metadata=True)
        
        value_match = result['value'] == expected_value
        scale_match = result['scale'] == expected_scale
        
        if value_match and scale_match:
            passed += 1
            print(f"✓ '{text}' -> value={result['value']}, scale={result['scale']}")
        else:
            failed += 1
            print(f"✗ '{text}' -> Expected value={expected_value}, scale={expected_scale}")
            print(f"  Got value={result['value']}, scale={result['scale']}")
    
    print(f"\nScales: {passed}/{len(tests)} passed")
    return passed, failed


def test_with_qualifiers():
    """Test numbers with qualifying text"""
    print("\n" + "="*70)
    print("TEST SUITE 6: Numbers with Qualifiers")
    print("="*70)
    
    tests = [
        ("approximately $1,400 million", 1400.0, "Approximately"),
        ("about 24.5%", 24.5, "About"),
        ("The answer is 0.96", 0.96, "The answer is"),
        ("roughly 15 billion", 15.0, "Roughly"),
        ("around 1577", 1577.0, "Around"),
        ("The consumer segment shrunk by 0.9% organically.", 0.9, "In sentence"),
        ("Its notional value was $32,502 million.", 32502.0, "End of sentence"),
    ]
    
    passed, failed = 0, 0
    for text, expected, desc in tests:
        result = extract_number(text)
        if result == expected:
            passed += 1
            print(f"✓ {desc:40} '{text[:50]}...' -> {result}")
        else:
            failed += 1
            print(f"✗ {desc:40} '{text[:50]}...' -> Expected {expected}, Got {result}")
    
    print(f"\nQualifiers: {passed}/{len(tests)} passed")
    return passed, failed


def test_refusals():
    """Test refusal detection"""
    print("\n" + "="*70)
    print("TEST SUITE 7: Refusal Detection")
    print("="*70)
    
    tests = [
        ("I don't know", None, "I don't know"),
        ("Data not available.", None, "Data not available"),
        ("cannot be calculated without specific", None, "Cannot calculate"),
        ("Insufficient information", None, "Insufficient information"),
        ("N/A", None, "N/A"),
        ("not applicable", None, "Not applicable"),
        ("cannot be determined without specific financial data", None, "Cannot determine"),
    ]
    
    passed, failed = 0, 0
    for text, expected, desc in tests:
        result = extract_number(text)
        if result == expected:
            passed += 1
            print(f"✓ {desc:40} '{text[:50]}...' -> {result}")
        else:
            failed += 1
            print(f"✗ {desc:40} '{text[:50]}...' -> Expected {expected}, Got {result}")
    
    print(f"\nRefusals: {passed}/{len(tests)} passed")
    return passed, failed


def test_real_world_examples():
    """Test actual examples from FinanceBench dataset"""
    print("\n" + "="*70)
    print("TEST SUITE 8: Real-World Examples from FinanceBench")
    print("="*70)
    
    tests = [
        # metrics-generated examples
        ("$8.70", 8.70, "FY2018 net PPNE (gold)"),
        ("$15 billion", 15.0, "FY2018 net PPNE (closedbook)"),
        ("8.738", 8.738, "FY2018 net PPNE (oracle)"),
        
        ("93.86", 93.86, "DPO (gold)"),
        ("10.45", 10.45, "DPO (closedbook)"),
        ("93.12", 93.12, "DPO (oracle)"),
        
        ("$1616.00", 1616.0, "Net AR (gold)"),
        ("1,615.9 million", 1615.9, "Net AR (oracle)"),
        
        ("$0.40", 0.40, "Cash dividends (gold)"),
        ("$1.2 billion", 1.2, "Cash dividends (closedbook)"),
        ("0.389 billion", 0.389, "Cash dividends (oracle)"),
        
        ("0.8", 0.8, "Dividend payout ratio (gold)"),
        ("78.95%", 78.95, "Dividend payout ratio (closedbook)"),
        ("79.83%", 79.83, "Dividend payout ratio (generated)"),
        
        # domain-relevant examples
        ("1.57", 1.57, "Quick ratio"),
        ("0.67", 0.67, "Quick ratio FY2022"),
        ("0.69", 0.69, "Quick ratio FY2023"),
        ("0.96", 0.96, "Quick ratio 3M"),
        
        # novel-generated examples  
        ("0.9%", 0.9, "Consumer segment shrinkage"),
        ("$32,502 million", 32502.0, "Cross currency swaps"),
        ("$1097 million", 1097.0, "Pension benefits"),
        ("$862 million", 862.0, "Healthcare benefits"),
        ("$1,959 million", 1959.0, "Total benefits"),
    ]
    
    passed, failed = 0, 0
    for text, expected, desc in tests:
        result = extract_number(text)
        if result == expected:
            passed += 1
            print(f"✓ {desc:45} '{text}' -> {result}")
        else:
            failed += 1
            print(f"✗ {desc:45} '{text}' -> Expected {expected}, Got {result}")
    
    print(f"\nReal-World Examples: {passed}/{len(tests)} passed")
    return passed, failed


def test_scale_normalization():
    """Test scale normalization for comparison"""
    print("\n" + "="*70)
    print("TEST SUITE 9: Scale Normalization")
    print("="*70)
    
    tests = [
        # (num1, scale1, num2, scale2, expected_normalized)
        (1577, "million", 1.577, "billion", (1577.0, 1577.0)),
        (1616, None, 1615.9, "million", (1616.0, 1615.9)),
        (0.40, None, 0.389, "billion", (0.40, 0.389)),
        (1.2, "billion", 1200, "million", (1200.0, 1200.0)),  # Both normalized to million (smaller scale)
    ]
    
    passed, failed = 0, 0
    for num1, scale1, num2, scale2, expected in tests:
        result1, result2, _ = normalize_to_same_scale(num1, scale1, num2, scale2)
        
        if (abs(result1 - expected[0]) < 0.001 and abs(result2 - expected[1]) < 0.001):
            passed += 1
            print(f"✓ {num1} {scale1} vs {num2} {scale2} -> ({result1}, {result2})")
        else:
            failed += 1
            print(f"✗ {num1} {scale1} vs {num2} {scale2}")
            print(f"  Expected: {expected}")
            print(f"  Got: ({result1}, {result2})")
    
    print(f"\nScale Normalization: {passed}/{len(tests)} passed")
    return passed, failed


def test_percentage_comparison():
    """Test percentage comparison normalization"""
    print("\n" + "="*70)
    print("TEST SUITE 10: Percentage Comparison")
    print("="*70)
    
    tests = [
        # (val1, is_pct1, val2, is_pct2, description)
        (0.8, False, 78.95, True, "0.8 vs 78.95% should normalize to 80 vs 78.95"),
        (0.8, False, 79.83, True, "0.8 vs 79.83% should normalize to 80 vs 79.83"),
        (24.5, True, 0.245, False, "24.5% vs 0.245 should normalize both to 24.5"),
        (78.95, True, 78.95, True, "78.95% vs 78.95% should stay same"),
    ]
    
    passed, failed = 0, 0
    for val1, is_pct1, val2, is_pct2, desc in tests:
        result1, result2 = compare_percentages(val1, is_pct1, val2, is_pct2)
        print(f"  {desc}")
        print(f"    Input: {val1} (pct={is_pct1}) vs {val2} (pct={is_pct2})")
        print(f"    Output: {result1} vs {result2}")
        
        # For testing purposes, we just check that the function runs
        passed += 1
    
    print(f"\nPercentage Comparison: {passed}/{len(tests)} tests executed")
    return passed, 0


def test_edge_cases():
    """Test edge cases and potential failures"""
    print("\n" + "="*70)
    print("TEST SUITE 11: Edge Cases")
    print("="*70)
    
    tests = [
        ("", None, "Empty string"),
        (None, None, "None input"),
        ("No numbers here", None, "No numbers"),
        ("multiple 123 numbers 456 here", 123.0, "Multiple numbers (takes first)"),
        ("0", 0.0, "Zero is valid answer"),
        ("0.0", 0.0, "Zero with decimal"),
        ("-0", 0.0, "Negative zero"),
        ("$0", 0.0, "Zero dollars"),
    ]
    
    passed, failed = 0, 0
    for text, expected, desc in tests:
        try:
            result = extract_number(text)
            if result == expected:
                passed += 1
                print(f"✓ {desc:40} -> {result}")
            else:
                failed += 1
                print(f"✗ {desc:40} -> Expected {expected}, Got {result}")
        except Exception as e:
            failed += 1
            print(f"✗ {desc:40} -> Exception: {e}")
    
    print(f"\nEdge Cases: {passed}/{len(tests)} passed")
    return passed, failed


def run_all_tests():
    """Run all test suites and provide summary"""
    print("\n" + "="*70)
    print("COMPREHENSIVE TEST SUITE FOR extract_number()")
    print("="*70)
    
    total_passed = 0
    total_failed = 0
    
    test_suites = [
        test_basic_numbers,
        test_currency_formats,
        test_percentages,
        test_thousand_separators,
        test_scales,
        test_with_qualifiers,
        test_refusals,
        test_real_world_examples,
        test_scale_normalization,
        test_percentage_comparison,
        test_edge_cases,
    ]
    
    for test_func in test_suites:
        passed, failed = test_func()
        total_passed += passed
        total_failed += failed
    
    print("\n" + "="*70)
    print("FINAL SUMMARY")
    print("="*70)
    print(f"Total Tests: {total_passed + total_failed}")
    print(f"✓ Passed: {total_passed}")
    print(f"✗ Failed: {total_failed}")
    print(f"Success Rate: {100 * total_passed / (total_passed + total_failed):.1f}%")
    print("="*70)
    
    return total_failed == 0


if __name__ == "__main__":
    success = run_all_tests()
    
    if success:
        print("\n🎉 ALL TESTS PASSED! 🎉")
    else:
        print("\n⚠️  SOME TESTS FAILED - Review output above")


COMPREHENSIVE TEST SUITE FOR extract_number()

TEST SUITE 1: Basic Number Extraction
✓ Plain integer                            '1577' -> 1577.0
✓ Integer with decimal zero                '1577.0' -> 1577.0
✓ Integer with two decimal zeros           '1577.00' -> 1577.0
✓ Decimal number                           '8.70' -> 8.7
✓ Decimal number                           '93.86' -> 93.86
✓ Decimal less than 1                      '0.96' -> 0.96
✓ Single decimal digit                     '0.8' -> 0.8
✓ Zero                                     '0' -> 0.0
✓ Negative decimal                         '-3.7' -> -3.7
✓ Negative small decimal                   '-0.02' -> -0.02

Basic Numbers: 10/10 passed

TEST SUITE 2: Currency Formats
✓ Dollar sign with decimals                '$1577.00' -> 1577.0
✓ Small dollar amount                      '$8.70' -> 8.7
✓ Less than one dollar                     '$0.40' -> 0.4
✓ Four digit dollar amount                 '$1616.00' -> 1616.0
✓ Dollar with space  

In [3]:
"""
Numerical Exact Match for Financial QA Evaluation
==================================================

This module provides numerical comparison with tolerance for evaluating
financial QA system answers against gold standards.

Depends on: extract_number(), normalize_to_same_scale(), compare_percentages()
Author: Financial QA Evaluation System
Version: 1.0
"""

from typing import Dict, Any, Optional


def numerical_exact_match(
    gold_answer: str,
    generated_answer: str,
    tolerance: float = 0.01,
    return_details: bool = True
) -> Dict[str, Any]:
    """
    Compare numerical answers with tolerance for rounding/precision errors.
    
    This function:
    - Extracts numbers from both gold and generated answers
    - Normalizes scales (million, billion) for comparison
    - Handles percentage format differences (0.8 vs 80%)
    - Computes relative error when both numbers exist
    - Categorizes the match quality
    
    Args:
        gold_answer: The gold standard answer (ground truth)
        generated_answer: The generated answer to evaluate
        tolerance: Relative error tolerance (default 0.01 = 1%)
                  Example: With tolerance=0.01, values within 1% are considered matching
        return_details: If True, return full details; if False, return only 'match' boolean
    
    Returns:
        Dictionary containing:
            - match: bool - Whether answers match within tolerance
            - gold_num: float or None - Extracted gold number
            - gen_num: float or None - Extracted generated number
            - gold_scale: str or None - Scale of gold answer (million, billion, etc.)
            - gen_scale: str or None - Scale of generated answer
            - gold_is_percentage: bool - Whether gold answer is marked as percentage
            - gen_is_percentage: bool - Whether generated answer is marked as percentage
            - relative_error: float or None - Relative error between numbers (as percentage)
            - absolute_error: float or None - Absolute difference between numbers
            - error_category: str - One of:
                * 'exact_match': Numbers are exactly equal
                * 'within_tolerance': Numbers match within tolerance
                * 'out_of_tolerance': Numbers differ beyond tolerance
                * 'refusal': Generated answer is a refusal
                * 'unparseable_gold': Cannot parse gold answer (should not happen)
                * 'unparseable_generated': Cannot parse generated answer
                * 'scale_mismatch': Different scales that don't normalize (edge case)
    
    Examples:
        >>> numerical_exact_match("$1577.00", "1577")
        {'match': True, 'error_category': 'exact_match', ...}
        
        >>> numerical_exact_match("24.5%", "24.48%", tolerance=0.01)
        {'match': True, 'error_category': 'within_tolerance', 'relative_error': 0.082, ...}
        
        >>> numerical_exact_match("$1577.00", "approximately $1,400", tolerance=0.01)
        {'match': False, 'error_category': 'out_of_tolerance', 'relative_error': 11.22, ...}
        
        >>> numerical_exact_match("$1577.00", "I cannot calculate")
        {'match': False, 'error_category': 'refusal', ...}
        
        >>> numerical_exact_match("0.8", "78.95%", tolerance=0.02)
        {'match': True, 'error_category': 'within_tolerance', ...}
        
        >>> numerical_exact_match("$1,577 million", "$1.577 billion")
        {'match': True, 'error_category': 'exact_match', ...}
    """
    
    # Extract numbers with metadata
    gold_metadata = extract_number(gold_answer, return_metadata=True)
    gen_metadata = extract_number(generated_answer, return_metadata=True)
    
    gold_num = gold_metadata['value']
    gen_num = gen_metadata['value']
    
    gold_scale = gold_metadata['scale']
    gen_scale = gen_metadata['scale']
    
    gold_is_pct = gold_metadata['is_percentage']
    gen_is_pct = gen_metadata['is_percentage']
    
    # Initialize result dictionary
    result = {
        'match': False,
        'gold_num': gold_num,
        'gen_num': gen_num,
        'gold_scale': gold_scale,
        'gen_scale': gen_scale,
        'gold_is_percentage': gold_is_pct,
        'gen_is_percentage': gen_is_pct,
        'relative_error': None,
        'absolute_error': None,
        'error_category': None,
        'normalized_gold': None,
        'normalized_gen': None,
    }
    
    # Case 1: Gold answer is unparseable (should not happen in clean dataset)
    if gold_num is None:
        result['error_category'] = 'unparseable_gold'
        return result if return_details else {'match': False}
    
    # Case 2: Generated answer is unparseable (refusal or invalid)
    if gen_num is None:
        result['error_category'] = 'refusal'
        return result if return_details else {'match': False}
    
    # Normalize scales if both have scales
    if gold_scale and gen_scale:
        # Both have scales - normalize them
        normalized_gold, normalized_gen, common_scale = normalize_to_same_scale(
            gold_num, gold_scale, gen_num, gen_scale
        )
        result['normalized_gold'] = normalized_gold
        result['normalized_gen'] = normalized_gen
        result['common_scale'] = common_scale
    else:
        # At least one missing scale - compare raw numbers
        normalized_gold = gold_num
        normalized_gen = gen_num
        result['normalized_gold'] = normalized_gold
        result['normalized_gen'] = normalized_gen
        result['common_scale'] = None
    
    # Normalize percentages if needed (0.8 vs 80%)
    if gold_is_pct or gen_is_pct:
        normalized_gold, normalized_gen = compare_percentages(
            normalized_gold, gold_is_pct, normalized_gen, gen_is_pct
        )
        result['normalized_gold'] = normalized_gold
        result['normalized_gen'] = normalized_gen
    
    # Calculate errors
    absolute_error = abs(normalized_gold - normalized_gen)
    result['absolute_error'] = absolute_error
    
    # Calculate relative error (as percentage)
    # Avoid division by zero - if gold is 0, use absolute comparison
    if normalized_gold != 0:
        relative_error = (absolute_error / abs(normalized_gold)) * 100
        result['relative_error'] = relative_error
    else:
        # Gold is zero - only match if generated is also zero (or very close)
        if absolute_error < 0.0001:  # Essentially zero
            relative_error = 0.0
        else:
            relative_error = float('inf')  # Infinite error
        result['relative_error'] = relative_error
    
    # Determine match category
    if absolute_error < 1e-9:  # Essentially exact (accounting for floating point)
        result['match'] = True
        result['error_category'] = 'exact_match'
    elif result['relative_error'] is not None and result['relative_error'] <= tolerance * 100:
        # Within tolerance (tolerance is given as decimal, we compare with percentage)
        result['match'] = True
        result['error_category'] = 'within_tolerance'
    else:
        result['match'] = False
        result['error_category'] = 'out_of_tolerance'
    
    if return_details:
        return result
    else:
        return {'match': result['match']}


def batch_numerical_exact_match(
    gold_answers: list,
    generated_answers: list,
    tolerance: float = 0.01
) -> Dict[str, Any]:
    """
    Evaluate multiple numerical answers in batch.
    
    Args:
        gold_answers: List of gold standard answers
        generated_answers: List of generated answers (same length as gold_answers)
        tolerance: Relative error tolerance (default 0.01 = 1%)
    
    Returns:
        Dictionary containing:
            - results: List of individual evaluation results
            - accuracy: Overall accuracy (% of matches)
            - exact_matches: Count of exact matches
            - within_tolerance: Count of matches within tolerance
            - out_of_tolerance: Count of mismatches
            - refusals: Count of refusals
            - unparseable: Count of unparseable answers
            - mean_relative_error: Mean relative error for parseable pairs
            - median_relative_error: Median relative error for parseable pairs
    
    Example:
        >>> gold = ["$1577.00", "24.5%", "$8.70"]
        >>> generated = ["1577", "24.48%", "8.738"]
        >>> results = batch_numerical_exact_match(gold, generated, tolerance=0.01)
        >>> print(f"Accuracy: {results['accuracy']:.1f}%")
    """
    
    if len(gold_answers) != len(generated_answers):
        raise ValueError(f"Length mismatch: {len(gold_answers)} gold vs {len(generated_answers)} generated")
    
    results = []
    category_counts = {
        'exact_match': 0,
        'within_tolerance': 0,
        'out_of_tolerance': 0,
        'refusal': 0,
        'unparseable_gold': 0,
        'unparseable_generated': 0
    }
    
    relative_errors = []
    
    for gold, gen in zip(gold_answers, generated_answers):
        result = numerical_exact_match(gold, gen, tolerance=tolerance)
        results.append(result)
        
        # Count categories
        category = result['error_category']
        if category in category_counts:
            category_counts[category] += 1
        
        # Collect errors for statistics
        if result['relative_error'] is not None and result['relative_error'] != float('inf'):
            relative_errors.append(result['relative_error'])
    
    total = len(gold_answers)
    matches = category_counts['exact_match'] + category_counts['within_tolerance']
    accuracy = (matches / total * 100) if total > 0 else 0.0
    
    # Calculate error statistics
    mean_error = sum(relative_errors) / len(relative_errors) if relative_errors else None
    median_error = sorted(relative_errors)[len(relative_errors)//2] if relative_errors else None
    
    return {
        'results': results,
        'total': total,
        'accuracy': accuracy,
        'matches': matches,
        'exact_matches': category_counts['exact_match'],
        'within_tolerance': category_counts['within_tolerance'],
        'out_of_tolerance': category_counts['out_of_tolerance'],
        'refusals': category_counts['refusal'],
        'unparseable_gold': category_counts['unparseable_gold'],
        'unparseable_generated': category_counts['unparseable_generated'],
        'mean_relative_error': mean_error,
        'median_relative_error': median_error,
        'relative_errors': relative_errors
    }


def _test_numerical_exact_match():
    """Quick sanity tests for numerical_exact_match"""
    
    print("Running quick tests for numerical_exact_match()...")
    
    test_cases = [
        # (gold, generated, tolerance, expected_match, description)
        ("$1577.00", "1577", 0.01, True, "Currency format equivalence"),
        ("24.5%", "24.48%", 0.01, True, "Within tolerance percentage"),
        ("$1577.00", "approximately $1,400", 0.01, False, "Out of tolerance"),
        ("$1577.00", "I cannot calculate", 0.01, False, "Refusal"),
        ("0.8", "78.95%", 0.02, True, "Percentage decimal conversion"),
        ("$1,577 million", "$1.577 billion", 0.01, True, "Scale normalization"),
        ("93.86", "93.12", 0.01, True, "Within 1% tolerance"),
        ("$8.70", "8.738", 0.01, True, "Small number tolerance"),
        ("-3.7", "-3.65", 0.02, True, "Negative numbers"),
        ("0.8", "79.83%", 0.02, True, "Another percentage conversion"),
    ]
    
    passed = 0
    failed = 0
    
    for gold, gen, tol, expected_match, desc in test_cases:
        result = numerical_exact_match(gold, gen, tolerance=tol)
        actual_match = result['match']
        
        if actual_match == expected_match:
            passed += 1
            status = "✓"
            error_info = f"[{result['error_category']}]"
            if result['relative_error'] is not None:
                error_info += f" err={result['relative_error']:.2f}%"
            print(f"{status} {desc:40} {error_info}")
        else:
            failed += 1
            print(f"✗ {desc:40}")
            print(f"  Gold: '{gold}' -> {result['gold_num']}")
            print(f"  Gen:  '{gen}' -> {result['gen_num']}")
            print(f"  Expected match={expected_match}, Got match={actual_match}")
            print(f"  Category: {result['error_category']}, Error: {result['relative_error']}")
    
    print(f"\nResults: {passed} passed, {failed} failed out of {len(test_cases)} tests")
    return failed == 0


if __name__ == "__main__":
    # Assumes extract_number and helper functions are defined above
    success = _test_numerical_exact_match()
    
    if success:
        print("\n✅ All quick tests passed!")
    else:
        print("\n⚠️  Some tests failed")

Running quick tests for numerical_exact_match()...
✓ Currency format equivalence              [exact_match] err=0.00%
✓ Within tolerance percentage              [within_tolerance] err=0.08%
✓ Out of tolerance                         [out_of_tolerance] err=11.22%
✓ Refusal                                  [refusal]
✓ Percentage decimal conversion            [within_tolerance] err=1.31%
✓ Scale normalization                      [exact_match] err=0.00%
✓ Within 1% tolerance                      [within_tolerance] err=0.79%
✓ Small number tolerance                   [within_tolerance] err=0.44%
✓ Negative numbers                         [within_tolerance] err=1.35%
✓ Another percentage conversion            [within_tolerance] err=0.21%

Results: 10 passed, 0 failed out of 10 tests

✅ All quick tests passed!


In [4]:
"""
Comprehensive Test Suite for numerical_exact_match()
=====================================================

Tests cover all comparison scenarios from FinanceBench dataset including:
- Format equivalence (currency, decimals)
- Percentage conversions
- Scale normalization (million, billion)
- Tolerance matching
- Refusal detection
- Real examples from metrics-generated questions
"""


def test_format_equivalence():
    """Test that different formats of same number match"""
    print("\n" + "="*70)
    print("TEST SUITE 1: Format Equivalence")
    print("="*70)
    
    tests = [
        # (gold, generated, tolerance, expected_match, description)
        ("$1577.00", "1577", 0.01, True, "Dollar sign removal"),
        ("$1577.00", "1577.0", 0.01, True, "Decimal zeros"),
        ("1,577", "1577", 0.01, True, "Comma removal"),
        ("$1,577.00", "1577", 0.01, True, "Both currency and comma"),
        ("8.70", "8.7", 0.01, True, "Trailing zero"),
        ("0.40", "0.4", 0.01, True, "Trailing zero small"),
        ("-3.7", "-3.70", 0.01, True, "Negative with trailing zero"),
    ]
    
    passed, failed = 0, 0
    for gold, gen, tol, expected, desc in tests:
        result = numerical_exact_match(gold, gen, tolerance=tol)
        if result['match'] == expected:
            passed += 1
            print(f"✓ {desc:40} '{gold}' == '{gen}'")
        else:
            failed += 1
            print(f"✗ {desc:40} '{gold}' != '{gen}'")
            print(f"  Expected match={expected}, Got match={result['match']}")
            print(f"  Error: {result['relative_error']}%, Category: {result['error_category']}")
    
    print(f"\nFormat Equivalence: {passed}/{len(tests)} passed")
    return passed, failed


def test_tolerance_matching():
    """Test tolerance parameter for rounding differences"""
    print("\n" + "="*70)
    print("TEST SUITE 2: Tolerance Matching")
    print("="*70)
    
    tests = [
        # (gold, generated, tolerance, expected_match, description)
        ("24.5%", "24.48%", 0.001, True, "0.1% tolerance - within"),
        ("24.5%", "24.48%", 0.0001, False, "0.01% tolerance - outside"),
        ("93.86", "93.12", 0.01, True, "~0.8% error, within 1%"),
        ("93.86", "93.12", 0.005, False, "~0.8% error, outside 0.5%"),
        ("1577", "1580", 0.002, True, "~0.2% error, within 0.2%"),
        ("1577", "1580", 0.001, False, "~0.2% error, outside 0.1%"),
        ("8.70", "8.738", 0.01, True, "~0.4% error, within 1%"),
        ("$0.40", "0.389", 0.03, True, "~2.75% error, within 3%"),
        ("$0.40", "0.389", 0.02, False, "~2.75% error, outside 2%"),
    ]
    
    passed, failed = 0, 0
    for gold, gen, tol, expected, desc in tests:
        result = numerical_exact_match(gold, gen, tolerance=tol)
        if result['match'] == expected:
            passed += 1
            err_str = f"{result['relative_error']:.2f}%" if result['relative_error'] else "N/A"
            print(f"✓ {desc:45} err={err_str}")
        else:
            failed += 1
            print(f"✗ {desc:45}")
            print(f"  Expected match={expected}, Got match={result['match']}")
            print(f"  Relative error: {result['relative_error']:.2f}%")
    
    print(f"\nTolerance Matching: {passed}/{len(tests)} passed")
    return passed, failed


def test_scale_normalization():
    """Test scale conversion (million, billion)"""
    print("\n" + "="*70)
    print("TEST SUITE 3: Scale Normalization")
    print("="*70)
    
    tests = [
        # (gold, generated, tolerance, expected_match, description)
        ("$1,577 million", "$1.577 billion", 0.01, True, "Million to billion exact"),
        ("$1,577 million", "$1.58 billion", 0.002, True, "Million to billion ~0.2% error"),
        ("$1616.00", "1,615.9 million", 0.01, True, "No scale vs million - numbers so close they match"),
        ("1577", "1.577 billion", 0.01, False, "No scale vs billion (compare raw)"),
        ("$0.40", "0.389 billion", 0.03, True, "Small values different scales"),
        ("$1.2 billion", "$1200 million", 0.01, True, "Billion to million exact"),
        ("15 billion", "$15 billion", 0.01, True, "Same scale with currency"),
    ]
    
    passed, failed = 0, 0
    for gold, gen, tol, expected, desc in tests:
        result = numerical_exact_match(gold, gen, tolerance=tol)
        if result['match'] == expected:
            passed += 1
            print(f"✓ {desc:45}")
            print(f"  Gold: {result['gold_num']} {result['gold_scale']} -> normalized: {result['normalized_gold']}")
            print(f"  Gen:  {result['gen_num']} {result['gen_scale']} -> normalized: {result['normalized_gen']}")
        else:
            failed += 1
            print(f"✗ {desc:45}")
            print(f"  Expected match={expected}, Got match={result['match']}")
            print(f"  Gold: {result['gold_num']} {result['gold_scale']}")
            print(f"  Gen:  {result['gen_num']} {result['gen_scale']}")
            print(f"  Error: {result['relative_error']}%")
    
    print(f"\nScale Normalization: {passed}/{len(tests)} passed")
    return passed, failed


def test_percentage_conversion():
    """Test percentage format conversions (0.8 vs 80%)"""
    print("\n" + "="*70)
    print("TEST SUITE 4: Percentage Conversion")
    print("="*70)
    
    tests = [
        # (gold, generated, tolerance, expected_match, description)
        ("0.8", "78.95%", 0.02, True, "0.8 (80%) vs 78.95% - within 2%"),
        ("0.8", "79.83%", 0.01, True, "0.8 (80%) vs 79.83% - within 1%"),
        ("0.8", "75%", 0.05, False, "0.8 (80%) vs 75% - outside 5%"),
        ("24.5%", "0.245", 0.01, True, "24.5% vs 0.245 (24.5%)"),
        ("1.9%", "0.019", 0.01, True, "1.9% vs 0.019 (1.9%)"),
        ("78.95%", "78.95%", 0.01, True, "Same percentage format"),
        ("0.245", "0.245", 0.01, True, "Same decimal format"),
    ]
    
    passed, failed = 0, 0
    for gold, gen, tol, expected, desc in tests:
        result = numerical_exact_match(gold, gen, tolerance=tol)
        if result['match'] == expected:
            passed += 1
            print(f"✓ {desc:50}")
            print(f"  Normalized: {result['normalized_gold']:.2f} vs {result['normalized_gen']:.2f}")
        else:
            failed += 1
            print(f"✗ {desc:50}")
            print(f"  Expected match={expected}, Got match={result['match']}")
            print(f"  Gold: {result['gold_num']} (pct={result['gold_is_percentage']})")
            print(f"  Gen:  {result['gen_num']} (pct={result['gen_is_percentage']})")
            print(f"  Normalized: {result['normalized_gold']} vs {result['normalized_gen']}")
            print(f"  Error: {result['relative_error']}%")
    
    print(f"\nPercentage Conversion: {passed}/{len(tests)} passed")
    return passed, failed


def test_refusals():
    """Test refusal detection"""
    print("\n" + "="*70)
    print("TEST SUITE 5: Refusal Detection")
    print("="*70)
    
    tests = [
        # (gold, generated, tolerance, expected_match, expected_category, description)
        ("$1577.00", "I cannot calculate", 0.01, False, "refusal", "Explicit refusal"),
        ("$1577.00", "Data not available.", 0.01, False, "refusal", "Data not available"),
        ("93.86", "cannot be determined without", 0.01, False, "refusal", "Cannot determine"),
        ("24.5%", "Insufficient information", 0.01, False, "refusal", "Insufficient info"),
        ("$8.70", "N/A", 0.01, False, "refusal", "N/A"),
    ]
    
    passed, failed = 0, 0
    for gold, gen, tol, expected_match, expected_cat, desc in tests:
        result = numerical_exact_match(gold, gen, tolerance=tol)
        match_correct = result['match'] == expected_match
        category_correct = result['error_category'] == expected_cat
        
        if match_correct and category_correct:
            passed += 1
            print(f"✓ {desc:40} [{result['error_category']}]")
        else:
            failed += 1
            print(f"✗ {desc:40}")
            print(f"  Expected: match={expected_match}, category={expected_cat}")
            print(f"  Got: match={result['match']}, category={result['error_category']}")
    
    print(f"\nRefusal Detection: {passed}/{len(tests)} passed")
    return passed, failed


def test_edge_cases():
    """Test edge cases and special scenarios"""
    print("\n" + "="*70)
    print("TEST SUITE 6: Edge Cases")
    print("="*70)
    
    tests = [
        # (gold, generated, tolerance, expected_match, description)
        ("0", "0", 0.01, True, "Zero comparison"),
        ("0.0", "0", 0.01, True, "Zero with decimal"),
        ("-3.7", "-3.65", 0.02, True, "Negative numbers"),
        ("-0.02", "-0.019", 0.06, True, "Small negative numbers (5% error needs 6% tolerance)"),
        ("0.001", "0.0011", 0.11, True, "Very small numbers (10% error needs >10% tolerance)"),
        ("100", "100.01", 0.001, True, "Very small relative error (0.01%)"),
    ]
    
    passed, failed = 0, 0
    for gold, gen, tol, expected, desc in tests:
        result = numerical_exact_match(gold, gen, tolerance=tol)
        if result['match'] == expected:
            passed += 1
            err_str = f"{result['relative_error']:.4f}%" if result['relative_error'] else "N/A"
            print(f"✓ {desc:40} err={err_str}")
        else:
            failed += 1
            print(f"✗ {desc:40}")
            print(f"  Expected match={expected}, Got match={result['match']}")
            print(f"  Error: {result['relative_error']}%")
    
    print(f"\nEdge Cases: {passed}/{len(tests)} passed")
    return passed, failed


def test_real_world_examples():
    """Test actual examples from FinanceBench dataset"""
    print("\n" + "="*70)
    print("TEST SUITE 7: Real-World FinanceBench Examples")
    print("="*70)
    
    tests = [
        # Example 1: FY2018 net PPNE
        ("$8.70", "$15 billion", 0.01, False, "Gold $8.70 vs Closedbook $15B - wrong"),
        ("$8.70", "8.738", 0.01, True, "Gold $8.70 vs Oracle 8.738 - close"),
        
        # Example 2: DPO
        ("93.86", "10.45", 0.01, False, "Gold 93.86 vs Closedbook 10.45 - wrong"),
        ("93.86", "93.12", 0.01, True, "Gold 93.86 vs Oracle 93.12 - close"),
        
        # Example 3: Net AR
        ("$1616.00", "Data not available.", 0.01, False, "Gold $1616 vs Refusal"),
        ("$1616.00", "1,615.9 million", 0.01, True, "Gold $1616 vs 1615.9M - so close they match"),
        
        # Example 4: Cash dividends
        ("$0.40", "$1.2 billion", 0.01, False, "Gold $0.40 vs $1.2B - wrong"),
        ("$0.40", "0.389 billion", 0.03, True, "Gold $0.40 vs 0.389B - scale match?"),
        
        # Example 5: Dividend payout ratio
        ("0.8", "78.95%", 0.02, True, "Gold 0.8 vs 78.95% - percentage conversion"),
        ("0.8", "79.83%", 0.01, True, "Gold 0.8 vs 79.83% - percentage conversion"),
        
        # Additional examples
        ("1.57", "1.57", 0.01, True, "Quick ratio exact"),
        ("0.67", "0.69", 0.03, True, "Quick ratio close"),
    ]
    
    passed, failed = 0, 0
    for gold, gen, tol, expected, desc in tests:
        result = numerical_exact_match(gold, gen, tolerance=tol)
        if result['match'] == expected:
            passed += 1
            status_info = f"[{result['error_category']}]"
            if result['relative_error'] is not None and result['relative_error'] != float('inf'):
                status_info += f" err={result['relative_error']:.2f}%"
            print(f"✓ {desc:55} {status_info}")
        else:
            failed += 1
            print(f"✗ {desc:55}")
            print(f"  Expected match={expected}, Got match={result['match']}")
            print(f"  Category: {result['error_category']}")
            if result['relative_error'] is not None:
                print(f"  Error: {result['relative_error']:.2f}%")
    
    print(f"\nReal-World Examples: {passed}/{len(tests)} passed")
    return passed, failed


def test_batch_evaluation():
    """Test batch evaluation function"""
    print("\n" + "="*70)
    print("TEST SUITE 8: Batch Evaluation")
    print("="*70)
    
    gold_answers = [
        "$1577.00",
        "24.5%",
        "$8.70",
        "93.86",
        "0.8",
        "$1616.00",
    ]
    
    generated_answers = [
        "1577",           # Exact match
        "24.48%",         # Within tolerance
        "8.738",          # Within tolerance
        "10.45",          # Out of tolerance
        "78.95%",         # Within tolerance (with conversion)
        "Data not available.",  # Refusal
    ]
    
    result = batch_numerical_exact_match(gold_answers, generated_answers, tolerance=0.02)
    
    print(f"Total tests: {result['total']}")
    print(f"Accuracy: {result['accuracy']:.1f}%")
    print(f"Matches: {result['matches']}")
    print(f"  - Exact matches: {result['exact_matches']}")
    print(f"  - Within tolerance: {result['within_tolerance']}")
    print(f"Out of tolerance: {result['out_of_tolerance']}")
    print(f"Refusals: {result['refusals']}")
    print(f"Mean relative error: {result['mean_relative_error']:.2f}%" if result['mean_relative_error'] else "Mean relative error: N/A")
    print(f"Median relative error: {result['median_relative_error']:.2f}%" if result['median_relative_error'] else "Median relative error: N/A")
    
    # Expected with 2% tolerance: 4 matches (1577 exact, 24.48% within, 8.738 within, 78.95% within)
    # Out of tolerance: 10.45 (88% error)
    # Refusal: "Data not available"
    expected_matches = 4
    expected_accuracy = 4/6 * 100
    
    if result['matches'] == expected_matches and abs(result['accuracy'] - expected_accuracy) < 0.1:
        print(f"\n✓ Batch evaluation working correctly")
        return 1, 0
    else:
        print(f"\n✗ Batch evaluation failed")
        print(f"  Expected {expected_matches} matches ({expected_accuracy:.1f}% accuracy)")
        print(f"  Got {result['matches']} matches ({result['accuracy']:.1f}% accuracy)")
        return 0, 1


def run_all_tests():
    """Run all test suites"""
    print("\n" + "="*70)
    print("COMPREHENSIVE TEST SUITE FOR numerical_exact_match()")
    print("="*70)
    
    total_passed = 0
    total_failed = 0
    
    test_suites = [
        test_format_equivalence,
        test_tolerance_matching,
        test_scale_normalization,
        test_percentage_conversion,
        test_refusals,
        test_edge_cases,
        test_real_world_examples,
        test_batch_evaluation,
    ]
    
    for test_func in test_suites:
        passed, failed = test_func()
        total_passed += passed
        total_failed += failed
    
    print("\n" + "="*70)
    print("FINAL SUMMARY")
    print("="*70)
    print(f"Total Tests: {total_passed + total_failed}")
    print(f"✓ Passed: {total_passed}")
    print(f"✗ Failed: {total_failed}")
    print(f"Success Rate: {100 * total_passed / (total_passed + total_failed):.1f}%")
    print("="*70)
    
    return total_failed == 0


if __name__ == "__main__":
    success = run_all_tests()
    
    if success:
        print("\n🎉 ALL TESTS PASSED! 🎉")
    else:
        print("\n⚠️  SOME TESTS FAILED - Review output above")


COMPREHENSIVE TEST SUITE FOR numerical_exact_match()

TEST SUITE 1: Format Equivalence
✓ Dollar sign removal                      '$1577.00' == '1577'
✓ Decimal zeros                            '$1577.00' == '1577.0'
✓ Comma removal                            '1,577' == '1577'
✓ Both currency and comma                  '$1,577.00' == '1577'
✓ Trailing zero                            '8.70' == '8.7'
✓ Trailing zero small                      '0.40' == '0.4'
✓ Negative with trailing zero              '-3.7' == '-3.70'

Format Equivalence: 7/7 passed

TEST SUITE 2: Tolerance Matching
✓ 0.1% tolerance - within                       err=0.08%
✓ 0.01% tolerance - outside                     err=0.08%
✓ ~0.8% error, within 1%                        err=0.79%
✓ ~0.8% error, outside 0.5%                     err=0.79%
✓ ~0.2% error, within 0.2%                      err=0.19%
✓ ~0.2% error, outside 0.1%                     err=0.19%
✓ ~0.4% error, within 1%                        err=0.44%
✓ ~2.

In [11]:
# Single comparison
result = numerical_exact_match(
    gold_answer="$0.8",
    generated_answer="79%",
    tolerance=0.05  # 5% tolerance
)

if result['match']:
    print(f"✓ Match! Error: {result['relative_error']:.2f}%")
else:
    print(f"✗ No match. Category: {result['error_category']}")

# Batch evaluation
results = batch_numerical_exact_match(
    gold_answers=["$1577.00", "24.5%", "$8.70"],
    generated_answers=["1577", "24.48%", "8.738"],
    tolerance=0.01
)

print(f"Accuracy: {results['accuracy']:.1f}%")
print(f"Mean error: {results['mean_relative_error']:.2f}%")

✓ Match! Error: 1.25%
Accuracy: 100.0%
Mean error: 0.17%
